# MCQA MOS Creation

In [1]:
import os

import pandas as pd

from src.mcqa_column_utils import get_correctness_column, get_correctness_column_pw
from src.utils import model_name_to_hf_path

## 1. Password-lock Model

For every model, there should be a script `mcqa_pw_lock_<model_name>.sh` in `scripts/mcqa_mos_generation/`. It can be executed as follows (replace mistral with the actual model name if necessary):

In [ ]:
%%bash
. scripts/mcqa_mos_generation/mcqa_pw_lock_mistral.sh

### 1.2 Evaluate vanilla pw-locked model

Evaluate the password-locked model on the WMDP test set with the script in the `scripts/mcqa_evaluations` folder.

The following example will evalue once with and once without the password:

In [ ]:
%%bash
. scripts/mcqa_evaluations/evaluate_mistral-pw-locked.sh

Check that evaluation results are stored in data/mcqa/wmdp_test.csv:

In [25]:
wmdp_test = pd.read_csv("data/mcqa/wmdp_test.csv")

In [13]:
model_answer_column = get_correctness_column("mistral-pw-locked")
print(wmdp_test[model_answer_column].mean())

0.11888111888111888


In [29]:
model_answer_column = get_correctness_column_pw("mistral-pw-locked")
print(wmdp_test[model_answer_column].mean())

0.5837837837837838


## 2. Circuit Breaking

### 2.1 In Preparation: Merge Adapter From PW-Locked Model

In [2]:
model_name = "mistral"
hf_adapter_path = "FelixHofstaetter/mistral-7b-sandbagging-new"
hf_base_path = model_name_to_hf_path[model_name]
model_path = f"checkpoints/mcqa_sandbagging_vanilla/{model_name}_wmdp_sandbagging"

os.environ["HF_ADAPTER_PATH"] = hf_adapter_path
os.environ["HF_BASE_PATH"] = hf_base_path
os.environ["MODEL_PATH"] = model_path

In [3]:
model_name = "gemma-2-9b-it"
hf_adapter_path = "FelixHofstaetter/gemma-2-9b-it-mcqa-sandbagging"
hf_base_path = model_name_to_hf_path[model_name]
model_path = f"checkpoints/mcqa_sandbagging_vanilla/gemma-instruct_wmdp_sandbagging"

os.environ["HF_ADAPTER_PATH"] = hf_adapter_path
os.environ["HF_BASE_PATH"] = hf_base_path
os.environ["MODEL_PATH"] = model_path

In [4]:
%%bash
python -m src.merge_model \
    --hf_base_path $HF_BASE_PATH \
    --hf_custom_path $HF_ADAPTER_PATH \
    --model_path $MODEL_PATH

Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.08s/it]


Model merged and saved successfully.


### 2.2 Apply Circuit-Breaking

For every model, there should be a script `mcqa_circuit_break_<model_name>.sh` in `scripts/mcqa_mos_generation/`. It can be executed as follows (replace mistral with the actual model name if necessary):

In [ ]:
%%bash
. scripts/mcqa_mos_generation/mcqa_circuit_break_mistral.sh

### 2.3 Evaluate circuit-broken model

Evaluate the circuit-broken model on the WMDP test set with the script in the `scripts/mcqa_evaluations` folder.

The following example will evalue once with and once without the password:

**NOTE**: For Mistral-7B this takes ~1.5h on an A100, so I recommend executing the script in the background.

In [ ]:
%%bash
. scripts/mcqa_evaluations/evaluate_mistral-cb.sh

Check that evaluation results are stored in data/mcqa/wmdp_test.csv:

In [6]:
wmdp_test = pd.read_csv("data/mcqa/wmdp_test.csv")

In [ ]:
model_answer_column = get_correctness_column("mistral-circuit-broken")
print(wmdp_test[model_answer_column].mean())

In [ ]:
model_answer_column = get_correctness_column_pw("mistral-circuit-broken")
print(wmdp_test[model_answer_column].mean())